In [ ]:
!pip install transformers==4.28.1

In [ ]:
!pip install datasets==2.12.0

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import csv

# Load data from CSV files (no headers)
def load_data(filename):
    with open(filename, 'r', encoding='utf-8') as file:
        reader = csv.reader(file)
        data = [(row[3], row[2]) for row in reader]
    return data

training_data = load_data('/content/twitter_training.csv')
validation_data = load_data('/content/twitter_validation.csv')

# Encode sentiment labels
le = LabelEncoder()
all_sentiments = [label for _, label in training_data] + [label for _, label in validation_data]
le.fit(all_sentiments)
training_labels = le.transform([label for _, label in training_data])
validation_labels = le.transform([label for _, label in validation_data])

# Load pre-trained model and tokenizer
model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=len(le.classes_))

# Tokenize data
def preprocess_function(examples):
    return tokenizer(examples, truncation=True, padding=True, return_tensors='pt')

training_texts = [text for text, _ in training_data]
validation_texts = [text for text, _ in validation_data]
tokenized_training_data = preprocess_function(training_texts)
tokenized_validation_data = preprocess_function(validation_texts)

# Create PyTorch datasets
class SentimentDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = SentimentDataset(tokenized_training_data, training_labels)
eval_dataset = SentimentDataset(tokenized_validation_data, validation_labels)

# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
)

# Create Trainer and train the model
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
)

trainer.train()

# Evaluate the model
trainer.evaluate()